# Dataracing Challenge 2023

## Predicting loan

In [ ]:
import pandas as pd
import random
import torch
import lightning
import optuna
import sklearn
import os
from sklearn.metrics import log_loss
from pandas import DataFrame
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from torch.utils.data.dataset import TensorDataset
from lightning.pytorch.trainer.trainer import Trainer
from lightning.pytorch.callbacks.early_stopping import EarlyStopping
from lightning.pytorch.callbacks.model_checkpoint import ModelCheckpoint
from lightning.pytorch.callbacks.prediction_writer import BasePredictionWriter
from lightning.pytorch import loggers as pl_loggers
from optuna.trial import Trial
from optuna.study import Study

In [ ]:
rng = random.Random("BL6ADS")

In [ ]:
# CONTRACT_ID,BORROWER_ID,CONTRACT_BANK_ID,CONTRACT_CREDIT_INTERMEDIARY,CONTRACT_CREDIT_LOSS,CONTRACT_CURRENCY,CONTRACT_DATE_OF_LOAN_AGREEMENT,CONTRACT_DEPT_SERVICE_TO_INCOME,CONTRACT_FREQUENCY_TYPE,CONTRACT_INCOME,CONTRACT_INSTALMENT_AMOUNT,CONTRACT_INSTALMENT_AMOUNT_2,CONTRACT_INTEREST_PERIOD,CONTRACT_INTEREST_RATE,CONTRACT_LGD,CONTRACT_LOAN_AMOUNT,CONTRACT_LOAN_CONTRACT_TYPE,CONTRACT_LOAN_TO_VALUE_RATIO,CONTRACT_LOAN_TYPE,CONTRACT_MARKET_VALUE,CONTRACT_MATURITY_DATE,CONTRACT_MORTGAGE_LENDING_VALUE,CONTRACT_MORTGAGE_TYPE,CONTRACT_REFINANCED,CONTRACT_RISK_WEIGHTED_ASSETS,CONTRACT_TYPE_OF_INTEREST_REPAYMENT,BORROWER_BIRTH_YEAR,BORROWER_CITIZENSHIP,BORROWER_COUNTRY,BORROWER_COUNTY,BORROWER_TYPE_OF_CUSTOMER,BORROWER_TYPE_OF_SETTLEMENT,TARGET_EVENT,TARGET_EVENT_DAY
# TpK8osXs,d8SqtuEV,1d42bbf5,2.0,0.0,31,2457052,,479a2e13,,9545.0,11059.0,,22.08,175424.0,192161,1,,69f70539,,2457798,,,1.0,1.0,,1217.0,98.0,98.0,20.0,A,,-,
# EtIEHrcH,lrdxML0g,1d42bbf5,,0.0,31,2457036,,479a2e13,,9528.0,,,16.57,994868.0,55590,2,,b503a0de,,2457217,784680.0,5.0,,74.17,,,,,,A,,-,
# 1G10DfKj,gII7nnq4,1d42bbf5,2.0,16350.0,31,2457043,7.05,479a2e13,127305.0,8899.0,8880.0,50.0,22.35,282553.0,275924,2,,b503a0de,,2458530,,,2.0,74.77,100003.0,1199.0,98.0,98.0,179.0,A,7.0,-,
# 2NLT774,MMkJ8z/e,1d42bbf5,,0.0,31,2457038,,479a2e13,,14329.0,12966.0,,0.0,68981.0,138862,2,,b503a0de,,2457345,,,,0.99,,1221.0,98.0,98.0,,A,,-,
# VpylRvay,M417onFP,1d42bbf5,2.0,2395.0,31,2457091,,479a2e13,,4003.0,3901.0,37.0,23.92,163319.0,169624,1,,69f70539,,2459648,,,2.0,74.3,100002.0,1260.0,98.0,98.0,178.0,A,1.0,-,
# 5tsDwTjS,u9GRAuen,1d42bbf5,2.0,0.0,31,2457093,,479a2e13,,11214.0,10727.0,,35.12,74244.0,100079,2,,b503a0de,,2457393,,,2.0,0.98,,1204.0,98.0,98.0,,A,,-,
# drZAWXaR,XPWFcJZI,1d42bbf5,,0.0,31,2457095,33.94,479a2e13,172647.0,10933.0,,,21.93,337632.0,338415,1,,69f70539,,2458556,,,,1.0,,,,,,A,,-,
# ebtK6u+,L2Nu2zZe,1d42bbf5,2.0,0.0,31,2457046,,479a2e13,,11405.0,11423.0,,16.28,118161.0,140975,2,,b503a0de,,2457562,,,2.0,1.03,,1255.0,98.0,98.0,4.0,A,,-,
# ffJh4MY,5A0GWOi2,1d42bbf5,2.0,0.0,31,2457049,,479a2e13,,6045.0,6008.0,,16.27,42047.0,60963,2,,b503a0de,,2457441,,,2.0,1.04,,1234.0,98.0,98.0,,A,,-,
# eo09vYuo,lqPGd2+x,1d42bbf5,2.0,0.0,31,2457081,,479a2e13,,10431.0,10420.0,,21.66,170739.0,190782,1,,69f70539,,2457814,,,2.0,1.0,0.0,1228.0,98.0,98.0,3.0,A,,-,
df = pd.read_csv('data/training_data.csv', sep=',', header=0, index_col=False)

# Print few rows
print(df.head())
# df.describe()

In [ ]:
# Transforms
borrower_ids = {value: idx for idx, value in enumerate(df['BORROWER_ID'].unique())}
borrower_ids_from_enum = {idx: value for idx, value in enumerate(df['BORROWER_ID'].unique())}

# CONTRACT_ID - Enum
def contract_id_enum_transform(df: DataFrame) -> DataFrame:
    contract_ids = {value: idx for idx, value in enumerate(df['CONTRACT_ID'].unique())}
    df['CONTRACT_ID'] = df['CONTRACT_ID'].map(contract_ids)
    return df

# BORROWER_ID - Enum
def borrower_id_enum_transform(df: DataFrame) -> DataFrame:
    df['BORROWER_ID'] = df['BORROWER_ID'].map(borrower_ids)
    return df

# CONTRACT_BANK_ID - Enum
def contract_bank_id_enum_transform(df: DataFrame) -> DataFrame:
    contract_bank_ids = {value: idx for idx, value in enumerate(df['CONTRACT_BANK_ID'].unique())}
    df['CONTRACT_BANK_ID'] = df['CONTRACT_BANK_ID'].map(contract_bank_ids)
    return df

# CONTRACT_CREDIT_INTERMEDIARY - Enum
def contract_credit_intermediary_enum_transform(df: DataFrame) -> DataFrame:
    contract_credit_intermediary_ids = {value: idx for idx, value in enumerate(df['CONTRACT_CREDIT_INTERMEDIARY'].unique())}
    df['CONTRACT_CREDIT_INTERMEDIARY'] = df['CONTRACT_CREDIT_INTERMEDIARY'].map(contract_credit_intermediary_ids)
    return df

# CONTRACT_CURRENCY - Enum
def contract_currency_enum_transform(df: DataFrame) -> DataFrame:
    contract_currency_ids = {value: idx for idx, value in enumerate(df['CONTRACT_CURRENCY'].unique())}
    df['CONTRACT_CURRENCY'] = df['CONTRACT_CURRENCY'].map(contract_currency_ids)
    return df

# CONTRACT_FREQUENCY_TYPE - Enum
def contract_frequency_type_enum_transform(df: DataFrame) -> DataFrame:
    contract_frequency_type_ids = {value: idx for idx, value in enumerate(df['CONTRACT_FREQUENCY_TYPE'].unique())}
    df['CONTRACT_FREQUENCY_TYPE'] = df['CONTRACT_FREQUENCY_TYPE'].map(contract_frequency_type_ids)
    return df

# CONTRACT_INCOME - Nan to 0
def contract_income_nan_to_zero_transform(df: DataFrame) -> DataFrame:
    df['CONTRACT_INCOME'] = df['CONTRACT_INCOME'].fillna(0)
    return df

# BORROWER_CITIZENSHIP - Float to Int
def borrower_citizenship_int_transform(df: DataFrame) -> DataFrame:
    df['BORROWER_CITIZENSHIP'] = df['BORROWER_CITIZENSHIP'].fillna(0).astype(int)
    return df

# BORROWER_COUNTRY - Float to Int
def borrower_country_int_transform(df: DataFrame) -> DataFrame:
    df['BORROWER_COUNTRY'] = df['BORROWER_COUNTRY'].fillna(0).astype(int)
    return df

# BORROWER_COUNTY - Float to Int
def borrower_county_int_transform(df: DataFrame) -> DataFrame:
    df['BORROWER_COUNTY'] = df['BORROWER_COUNTY'].fillna(0).astype(int)
    return df

# BORROWER_TYPE_OF_CUSTOMER - Enum
def borrower_type_of_customer_enum_transform(df: DataFrame) -> DataFrame:
    borrower_type_of_customer_ids = {value: idx for idx, value in enumerate(df['BORROWER_TYPE_OF_CUSTOMER'].unique())}
    df['BORROWER_TYPE_OF_CUSTOMER'] = df['BORROWER_TYPE_OF_CUSTOMER'].map(borrower_type_of_customer_ids)
    return df

# BORROWER_TYPE_OF_SETTLEMENT - Enum
def borrower_type_of_settlement_enum_transform(df: DataFrame) -> DataFrame:
    borrower_type_of_settlement_ids = {value: idx for idx, value in enumerate(df['BORROWER_TYPE_OF_SETTLEMENT'].unique())}
    df['BORROWER_TYPE_OF_SETTLEMENT'] = df['BORROWER_TYPE_OF_SETTLEMENT'].map(borrower_type_of_settlement_ids)
    return df

# TARGET_EVENT - Bool (1 if 'K', 0 otherwise)
def target_event_bool_transform(df: DataFrame) -> DataFrame:
    df['TARGET_EVENT'] = df['TARGET_EVENT'].map({'K': True, '': False, 'E': False}).astype(bool)
    return df

def df_to_32_bit_numeric_transform(df: DataFrame) -> DataFrame:
    df = df.fillna(0.0)
    for column in df.columns:
        if df[column].dtype == pd.StringDtype:
            # Enum transform
            column_values = {value: idx for idx, value in enumerate(df[column].unique())}
            df[column] = df[column].map(column_values).astype(pd.Int32Dtype())
        if df[column].dtype == pd.Float64Dtype:
            df[column] = df[column].astype(pd.Float32Dtype())
        if df[column].dtype == pd.Int64Dtype:
            df[column] = df[column].astype(pd.Int32Dtype())
    return df

def apply_transformations(tfs: list, df: DataFrame) -> DataFrame:
    for tf in tfs:
        df = tf(df)
    return df

In [ ]:
original_df = df.copy()
tfs = [
    # contract_id_enum_transform,
    # borrower_id_enum_transform,
    contract_bank_id_enum_transform,
    contract_credit_intermediary_enum_transform,
    contract_currency_enum_transform,
    contract_frequency_type_enum_transform,
    contract_income_nan_to_zero_transform,
    borrower_citizenship_int_transform,
    borrower_country_int_transform,
    borrower_county_int_transform,
    borrower_type_of_customer_enum_transform,
    borrower_type_of_settlement_enum_transform,
    target_event_bool_transform,
    df_to_32_bit_numeric_transform
]

df = apply_transformations(tfs, original_df.copy())

def get_train_test_split(df: DataFrame, train_size=40000, test_size=10000) -> list[DataFrame]:
    return train_test_split(
        df.drop(columns=['CONTRACT_ID', 'BORROWER_ID', 'TARGET_EVENT', 'TARGET_EVENT_DAY']), # Drop target column from training data
        df['TARGET_EVENT'], # Target column
        random_state=42,
        train_size=40000,
        test_size=10000)

train_x, test_x, train_y, test_y = get_train_test_split(df)
train_x_float = train_x.astype(float)
test_x_float = test_x.astype(float)
train_y_float = train_y.astype(float)
test_y_float = test_y.astype(float)

In [ ]:
from lightning.pytorch.utilities.types import EVAL_DATALOADERS


class PropertyDataModule(lightning.LightningDataModule):
    def __init__(self, train_x, test_x, train_y, test_y, batch_size: int):
        super().__init__()
        self.batch_size = batch_size
        self.train_x = train_x
        self.test_x = test_x
        self.train_y = train_y
        self.test_y = test_y

    def prepare_data(self) -> None:
        self.train_x_tensor = torch.tensor(self.train_x.values, dtype=torch.float32)
        self.test_x_tensor = torch.tensor(self.test_x.values, dtype=torch.float32)
        self.train_y_tensor = torch.tensor(self.train_y.values, dtype=torch.float32)
        self.test_y_tensor = torch.tensor(self.test_y.values, dtype=torch.float32)
        self.train_dataset = TensorDataset(self.train_x_tensor, self.train_y_tensor)
        self.test_dataset = TensorDataset(self.test_x_tensor, self.test_y_tensor)

    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True, pin_memory=True) # num_workers=os.cpu_count()

    def val_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=self.batch_size, shuffle=False, pin_memory=True) # num_workers=os.cpu_count()

    def test_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=self.batch_size, shuffle=False, pin_memory=True) # num_workers=os.cpu_count()

    def predict_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=self.batch_size, shuffle=False, pin_memory=True) # num_workers=os.cpu_count()

In [ ]:
from typing import Any


class DeepNN(lightning.LightningModule):
    def __init__(
        self,
        input_size: int,
        output_size: int,
        hidden_size: int,
        dropout: float,
        learning_rate: float):
        super().__init__()
        self.save_hyperparameters()

        self.input_size = input_size
        self.output_size = output_size
        self.hidden_size = hidden_size
        self.dropout = dropout
        self.learning_rate = learning_rate

        self.net = torch.nn.Sequential(
            torch.nn.Linear(self.input_size, self.hidden_size),
            torch.nn.ReLU(),
            torch.nn.Dropout(self.dropout),
            torch.nn.Linear(self.hidden_size, self.output_size),
            torch.nn.Sigmoid()
        )

    def forward(self, x):
        return self.net(x)

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.learning_rate)

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.net(x)
        y_hat = y_hat.squeeze()
        loss = torch.nn.functional.binary_cross_entropy(y_hat, y)
        self.log('train_loss', loss)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.net(x)
        y_hat = y_hat.squeeze()
        loss = torch.nn.functional.binary_cross_entropy(y_hat, y)
        self.log('val_loss', loss)
        return loss

    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.net(x)
        y_hat = y_hat.squeeze()
        loss = torch.nn.functional.binary_cross_entropy(y_hat, y)
        self.log('test_loss', loss)
        return loss

    def predict_step(self, batch, batch_idx, dataloader_idx=0):
        x, y = batch
        y_hat = self.net(x)
        y_hat = y_hat.squeeze()
        return y_hat

In [ ]:
data_module = PropertyDataModule(batch_size=64, train_x=train_x_float, test_x=test_x_float, train_y=train_y_float, test_y=test_y_float)

def trial_run(trial: Trial) -> float:
    early_stop_callback = EarlyStopping(
        monitor='val_loss',
        min_delta=0.001,
        patience=8,
        verbose=True,
        mode='min'
    )
    checkpoint_callback = ModelCheckpoint(
        monitor='val_loss',
        dirpath='models/',
        filename='loan_ml_prediction-{epoch}-{val_loss:.2f}',
        save_top_k=3,
        mode='min',

    )
    logger = pl_loggers.TensorBoardLogger('logs/', name='loan_ml_prediction')
    net = DeepNN(
        input_size=train_x.shape[1],
        output_size=1,
        hidden_size=trial.suggest_int('hidden_size', 2, 40, log=True),
        dropout=trial.suggest_float('dropout', 0.0, 0.5),
        learning_rate=trial.suggest_float('learning_rate', 1e-4, 1e-2, log=True),
    )
    trainer = Trainer(
        max_epochs=40,
        accelerator='gpu',
        callbacks=[early_stop_callback, checkpoint_callback],
        logger=logger,
    )
    trainer.fit(net, data_module)
    trainer.test(net, datamodule=data_module)
    loss = trainer.callback_metrics['test_loss'].item()
    trial.set_user_attr('test_loss', loss)
    return loss

In [ ]:
study = optuna.create_study(
    direction='minimize',
    pruner=optuna.pruners.MedianPruner(n_startup_trials=5, n_warmup_steps=30, interval_steps=10),
    sampler=optuna.samplers.TPESampler(multivariate=True, seed=42)
    )
study.optimize(trial_run, n_trials=10)
print(f'Best trial: {study.best_trial.value}')

In [ ]:
def flatten_list(list_of_lists, flat_list=None):
    if not flat_list:
        flat_list = []
    if not list_of_lists:
        return flat_list
    else:
        for item in list_of_lists:
            if type(item) == list:
                flatten_list(item, flat_list)
            else:
                flat_list.append(item)

    return flat_list

In [ ]:
# Get BORROWER_ID and keep only the rows in df which contains the BORROWER_ID from df_submission
df_submission = pd.read_csv('data/data_submission_example.csv', sep=',', header=0, index_col=False)

In [ ]:
# Number of unique TARGET_EVENT values and their count
df = original_df.copy()
print(f'Target event unique values', df['TARGET_EVENT'].unique())
print(f'Target event unique values count', df['TARGET_EVENT'].value_counts())

In [ ]:
df = original_df.copy()
print(f'Shared BORROWER_ID elements between df_submission and df', len(set(df_submission['BORROWER_ID'].unique()).intersection(df['BORROWER_ID'].unique())))
# Keep only the rows in df which contains the BORROWER_ID from df_submission
df = df[df['BORROWER_ID'].isin(df_submission['BORROWER_ID'].unique())]
print(f'Shared BORROWER_ID elements between df_submission and df after filtering', len(df))
print(f'Number of unique BORROWER_ID in df', len(df['BORROWER_ID'].unique()))

In [ ]:
df = original_df.copy()
# Keep only the rows in df which contains the BORROWER_ID from df_submission
df = df[df['BORROWER_ID'].isin(df_submission['BORROWER_ID'].unique())]

# df = df[df['BORROWER_ID'].isin(df_submission['BORROWER_ID'])]
# print(df.head())
# number of rows in df where TARGET_EVENT is NaN
# df = df[df['TARGET_EVENT'] != '-']
# print(f'DF without target event length', len(df[df['TARGET_EVENT'] != '-']))
print(f'DF length', len(df))
# number of rows in df_submission
print(f'DF Submission length', len(df_submission))

borrower_ids_from_enum = {idx: value for idx, value in enumerate(df['BORROWER_ID'].unique())}
df = apply_transformations(tfs, df)

val_x = df.drop(columns=['CONTRACT_ID', 'BORROWER_ID', 'TARGET_EVENT', 'TARGET_EVENT_DAY']) # Drop target column from training data
val_y = df['TARGET_EVENT'] # Target column

val_x_float = val_x.astype(float)
val_y_float = val_y.astype(float)

val_data_module = PropertyDataModule(batch_size=32, train_x=val_x_float, test_x=val_x_float, train_y=val_y, test_y=val_y)

checkpoint_path = None
# Load lowest loss checkpoint (dirname = models, filename = loan_ml_prediction-{epoch}-{val_loss:.2f})
for file in os.listdir('models/'):
    if file.startswith('loan_ml_prediction') and file.endswith('.ckpt'):
        # Check if loss is lower than current checkpoint
        if checkpoint_path is None:
            checkpoint_path = f'models/{file}'
            continue
        # loan_ml_prediction-epoch=0-val_loss=2.59-v5.ckpt
        # loan_ml_prediction-epoch=0-val_loss=2.59.ckpt
        # Get the 2.59 as float
        file_loss = float(file.removesuffix('.ckpt').split('=')[2].split('-')[0])
        current_loss = float(checkpoint_path.removesuffix('.ckpt').split('=')[2].split('-')[0])
        if float(file_loss < current_loss):
            checkpoint_path = f'models/{file}'

if checkpoint_path is None:
    raise Exception('No checkpoint found')
net = DeepNN.load_from_checkpoint(checkpoint_path)

df_submission = pd.read_csv('data/data_submission_example.csv', sep=',', header=0, index_col=False)

class CustomWriter(BasePredictionWriter):
    def __init__(self, output_dir, write_interval):
        super().__init__(write_interval)
        self.output_dir = output_dir

    def write_on_epoch_end(self, trainer, pl_module, predictions, batch_indices):
        global df_submission
        global df
        # print(f'Raw Predictions', predictions)
        # Predictions is a list of tensors, make it a single list
        # Last is a 0 dim tensor, make it a list first
        predictions = [p.tolist() for p in predictions]
        predictions = [element for sublist in predictions for element in (sublist if isinstance(sublist, list) else [sublist])]
        # New column in df: PRED with predictions
        df['PRED'] = predictions
        # Group by BORROWER_ID and get the mean of PRED
        # Need to subtract PRED from 1
        unique_preds = 1 - df.groupby('BORROWER_ID')['PRED'].mean()
        unique_borrowers = df['BORROWER_ID'].map(borrower_ids_from_enum).unique()
        df = pd.DataFrame({'BORROWER_ID': unique_borrowers, 'PRED': unique_preds})
        df.describe()
        # Keep rows which appear in BORROWER_ID from df_submission
        # df = df[df['BORROWER_ID'].isin(df_submission['BORROWER_ID'].unique())]
        df.to_csv('data/predictions.csv', index=False)
        # Keep only BORROWER_ID and PRED columns
        # print(f'Submissions DataFrame size', len(df_submission))

pred_writer = CustomWriter(output_dir="pred_path", write_interval="epoch")
trainer = Trainer(accelerator='gpu', callbacks=[pred_writer])
predictions = trainer.predict(net, datamodule=val_data_module, return_predictions=True)
# predictions = trainer.pred
# print(predictions)

# if len(predicted_values) != len(df_submission):
#     raise ValueError("Length of predictions does not match length of submission DataFrame.")

# df_submission['PRED'] = predicted_values
# df_submission.to_csv('data/predictions.csv', index=False)